# Import and Load Data

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import meterdatalogic as ml
import numpy as np

In [2]:
TZ = "Australia/Brisbane"

# 7 days of half-hourly data for one NMI
rng = pd.date_range("2025-01-01", periods=48*7, freq="30min", tz=TZ)
raw = pd.DataFrame({
    "t_start": rng,
    "nmi": "Q1234567890",
    "channel": "E1",
    "kwh": 0.5
})
raw.head()

df = ml.ingest.from_dataframe(raw, tz=TZ)
ml.validate.assert_canon(df)

In [3]:
# df = ml.ingest.from_nem12("../data/unzipped/Example_NEM12_month_solar.csv", tz="Australia/Brisbane")
df = ml.ingest.from_nem12("../data/Sample1.csv", tz="Australia/Brisbane")
ml.validate.assert_canon(df)

print(df.index.tz, df.index.name)
print(df.head())
print(df["flow"].value_counts())
df
# Shared ToU bands for examples
bands = [
    {"name": "off", "start": "00:00", "end": "16:00"},
    {"name": "peak", "start": "16:00", "end": "21:00"},
    {"name": "shoulder", "start": "21:00", "end": "24:00"},
]


Australia/Brisbane t_start
                               nmi channel         flow    kwh  cadence_min
t_start                                                                    
2014-05-06 00:00:00+10:00  SAMPLE1      E1  grid_import  0.144           30
2014-05-06 00:30:00+10:00  SAMPLE1      E1  grid_import  0.163           30
2014-05-06 01:00:00+10:00  SAMPLE1      E1  grid_import  0.106           30
2014-05-06 01:30:00+10:00  SAMPLE1      E1  grid_import  0.144           30
2014-05-06 02:00:00+10:00  SAMPLE1      E1  grid_import  0.169           30
flow
grid_import    35136
Name: count, dtype: int64


# Visualisations

In [4]:
hist = (
    df.reset_index()[["t_start","flow","kwh"]]
      .sort_values("t_start")
)

fig = px.area(
    hist, x="t_start", y="kwh", color="flow",
    title="Historical Interval Energy (kWh) by Flow",
    labels={"t_start":"Time", "kwh":"kWh"}
)
fig.update_layout(legend_title_text="Flow", hovermode="x unified")
fig.show()


In [5]:
daily = ml.transform.groupby_day(df).reset_index().melt(id_vars="day", var_name="flow", value_name="kwh")

fig = px.bar(
    daily, x="day", y="kwh", color="flow",
    title="Daily Energy by Flow",
    labels={"day":"Day","kwh":"kWh"}
)
fig.update_layout(hovermode="x unified")
fig.show()


In [6]:
monthly = ml.transform.groupby_month(df)  # columns: month + flows
melted = monthly.melt(id_vars="month", var_name="flow", value_name="kwh")

fig = px.bar(
    melted, x="month", y="kwh", color="flow",
    title="Monthly Energy by Flow",
    labels={"month":"Month","kwh":"kWh"},
    barmode="stack"
)
fig.update_layout(hovermode="x unified")
fig.show()


In [7]:
monthly = ml.transform.groupby_month(df)  # columns: month + flows
melted = monthly.melt(id_vars="month", var_name="flow", value_name="kwh")

fig = px.bar(
    melted, x="month", y="kwh", color="flow",
    title="Monthly Energy by Flow",
    labels={"month":"Month","kwh":"kWh"},
    barmode="stack"
)
fig.update_layout(hovermode="x unified")
fig.show()


In [8]:
prof = ml.transform.profile24(df)  # columns: slot + flows
melted = prof.melt(id_vars="slot", var_name="flow", value_name="kwh")

fig = px.line(
    melted, x="slot", y="kwh", color="flow",
    title="Average 24-Hour Profile (kWh per interval)",
    labels={"slot":"Time of Day","kwh":"kWh per interval"}
)
fig.update_xaxes(type="category", tickangle=-45)
fig.update_layout(hovermode="x unified")
fig.show()


In [9]:
# # Build a band label per timestamp
# def assign_tou_band(index, bands):
#     # Reuse the same logic as tou_bins: parse safely & wrap ranges
#     from pandas import Series, to_datetime
#     def parse(t): return to_datetime("00:00" if t=="24:00" else t, format="%H:%M").time()
#     def in_range(t, a, b): return (t>=a)&(t<b) if a<b else ((t>=a)|(t<b))

#     times = pd.Series(index.tz_convert(index.tz).time, index=index)
#     lab = pd.Series(index=index, dtype="object")
#     for b in bands:
#         a, z = parse(b["start"]), parse(b["end"])
#         mask = in_range(times, a, z)
#         lab[mask] = b["name"]
#     return lab.fillna("unassigned")

# ts = df.reset_index()[["t_start","kwh","flow"]].copy()
# ts["band"] = assign_tou_band(ts["t_start"], bands)

# fig = px.scatter(
#     ts, x="t_start", y="kwh", color="band",
#     title="Historical Energy with ToU Band Overlay",
#     labels={"t_start":"Time","kwh":"kWh"}
# )
# # Optional: add a faint area by flow behind the dots
# area = px.area(ts, x="t_start", y="kwh", color="flow").data
# for tr in area:
#     tr.opacity = 0.2
#     fig.add_trace(tr)
# fig.update_layout(hovermode="x unified")
# fig.show()


In [10]:
tou = ml.transform.tou_bins(df[df["flow"]=="grid_import"], bands)  # month + off/peak/shoulder
melted = tou.melt(id_vars="month", var_name="band", value_name="kwh")

fig = px.bar(
    melted, x="month", y="kwh", color="band",
    title="Monthly Import by ToU Band",
    labels={"month":"Month","kwh":"kWh"},
    barmode="stack"
)
fig.update_layout(hovermode="x unified")
fig.show()


In [11]:
demand = ml.transform.demand_window(df, start="16:00", end="21:00", days="MF")  # month, demand_kw

fig = px.bar(demand, x="month", y="demand_kw",
             title="Monthly Peak Demand (kW) in Window (MF 16:00–21:00)",
             labels={"month":"Month","demand_kw":"kW"})
fig.update_layout(hovermode="x unified")
fig.show()


In [12]:
plan = ml.types.Plan(
    usage_bands=[
        ml.types.ToUBand("off", "00:00", "16:00", 22.0),
        ml.types.ToUBand("peak","16:00", "21:00", 45.0),
        ml.types.ToUBand("shoulder","21:00", "24:00", 28.0),
    ],
    demand=ml.types.DemandCharge("16:00","21:00","MF", 12.0),
    fixed_c_per_day=95.0,
    feed_in_c_per_kwh=6.0
)

cost = ml.pricing.estimate_monthly_cost(df, plan)  # month, components, total
melted = cost.melt(id_vars="month", value_vars=["energy_cost","demand_cost","fixed_cost","feed_in_credit"],
                   var_name="component", value_name="AUD")

fig = px.bar(
    melted, x="month", y="AUD", color="component",
    title="Monthly Cost Breakdown",
    labels={"month":"Month","AUD":"$"},
    barmode="relative"  # feed_in_credit is negative → subtracts
)
# Optional: add total as line
fig2 = go.Figure(fig.data)
fig2.add_trace(go.Scatter(x=cost["month"], y=cost["total"], name="Total", mode="lines+markers"))
fig2.update_layout(title="Monthly Cost Breakdown (+ Total)", hovermode="x unified")
fig2.show()


In [13]:
g = df.copy()
g["weekday"] = g.index.weekday  # 0=Mon
g["hour"] = g.index.hour
heat = g.groupby(["weekday","hour"])["kwh"].mean().reset_index()

fig = px.density_heatmap(
    heat, x="hour", y="weekday", z="kwh",
    title="Average kWh by Hour × Weekday",
    labels={"hour":"Hour of Day","weekday":"Day (0=Mon)","kwh":"kWh per interval"},
    nbinsx=24, nbinsy=7, histfunc="avg", color_continuous_scale="Viridis"
)
fig.update_yaxes(dtick=1)
fig.show()


# Scenario Modelling

In [28]:
import pandas as pd
import meterdatalogic as ml
from meterdatalogic.scenario import EVConfig, PVConfig, BatteryConfig, run

# Plan for costs (optional)
plan = ml.types.Plan(
    usage_bands=[
        ml.types.ToUBand("off","00:00","16:00",22.0),
        ml.types.ToUBand("peak","16:00","21:00",45.0),
        ml.types.ToUBand("shoulder","21:00","24:00",28.0),
    ],
    demand=ml.types.DemandCharge("16:00","21:00","MF",12.0),
    fixed_c_per_day=95.0,
    feed_in_c_per_kwh=6.0
)

# Scenario configs
ev_cfg = EVConfig(daily_kwh=8.0, max_kw=7.0, window_start="18:00", window_end="22:00", days="ALL", strategy="immediate")
pv_cfg = PVConfig(system_kwp=6.6, inverter_kw=5.0, loss_fraction=0.15)
bat_cfg = BatteryConfig(capacity_kwh=10.0, max_kw=5.0, round_trip_eff=0.9, soc_min=0.1, soc_max=0.95)

# Run scenario
result = run(df, ev=ev_cfg, pv=pv_cfg, battery=bat_cfg, plan=plan)

print("Δ Import (kWh):", round(result.delta["import_kwh_delta"], 2))
print("Δ Export (kWh):", round(result.delta["export_kwh_delta"], 2))
print("Δ Cost ($):", None if result.delta["cost_total_delta"] is None else round(result.delta["cost_total_delta"], 2))
print("Explain:", {k: round(v,2) if isinstance(v,(int,float)) and v is not None else v for k,v in result.explain.items()})

Δ Import (kWh): -9527.71
Δ Export (kWh): 19145.37
Δ Cost ($): -3135.8
Explain: {'ev_kwh': 5856.0, 'pv_kwh': 28601.06, 'battery_discharge_kwh': 5928.01, 'battery_charge_kwh': 6586.68, 'battery_cycles_est': 592.8, 'pv_self_consumption_pct': 33.06}


In [29]:
# Optional: charts (Plotly)
import plotly.express as px

before_m = ml.transform.groupby_month(result.df_before).melt(id_vars="month", var_name="flow", value_name="kwh")
after_m = ml.transform.groupby_month(result.df_after).melt(id_vars="month", var_name="flow", value_name="kwh")
before_m["scenario"] = "before"; after_m["scenario"] = "after"
both = pd.concat([before_m, after_m], ignore_index=True)

fig = px.bar(both, x="month", y="kwh", color="flow", facet_row="scenario",
             barmode="stack", title="Monthly Energy by Flow — Before vs After")
fig.show()


In [30]:
# Average 24-hour profile: baseline vs scenario (overlayed)
# — shows import/export as separate traces, plus an optional "net" line.

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

try:
    import meterlogic as ml
except ImportError:
    import meterdatalogic as ml

def plot_avg_profile_before_after(df_before: pd.DataFrame,
                                  df_after: pd.DataFrame,
                                  *,
                                  show_net: bool = True,
                                  title: str = "Average 24-Hour Profile — Before vs After"):
    """
    Overlays baseline and scenario average day profiles.
    - Uses ml.transform.profile24() to compute kWh per interval slot (mean across days)
    - Plots grid_import and grid_export_solar as separate traces
    - Optionally adds a 'net' line (import - export) for each scenario
    """
    # Compute profile (slot + columns per flow)
    prof_b = ml.transform.profile24(df_before)  # columns: slot, grid_import, grid_export_solar, ...
    prof_a = ml.transform.profile24(df_after)

    # Ensure both have the same slot ordering (categorical 00:00..23:30)
    slots = prof_b["slot"].unique().tolist()
    prof_a = prof_a.set_index("slot").reindex(slots).reset_index()

    # Helper to build a tidy frame with optional net
    def melt_with_net(prof: pd.DataFrame, label: str) -> pd.DataFrame:
        flows = [c for c in prof.columns if c != "slot"]
        tidy = prof.melt(id_vars="slot", value_vars=flows, var_name="flow", value_name="kwh")
        tidy["scenario"] = label

        if show_net:
            imp = prof.get("grid_import", pd.Series(0.0, index=prof.index))
            exp = prof.get("grid_export_solar", pd.Series(0.0, index=prof.index))
            net = imp - exp  # positive = net import, negative = net export
            tidy = pd.concat([
                tidy,
                pd.DataFrame({"slot": prof["slot"], "flow": "net", "kwh": net, "scenario": label})
            ], ignore_index=True)

        return tidy

    tb = melt_with_net(prof_b, "before")
    ta = melt_with_net(prof_a, "after")
    both = pd.concat([tb, ta], ignore_index=True)

    # Optional: flip export sign so export appears below zero (purely visual)
    both["kwh_plot"] = np.where(both["flow"].str.contains("export"), -both["kwh"], both["kwh"])

    # Build the figure: overlay 'before' and 'after'
    fig = go.Figure()

    def add_lines(df_scn, name_suffix, dash=None, width=2):
        for flow_name in sorted(df_scn["flow"].unique()):
            sub = df_scn[df_scn["flow"] == flow_name]
            fig.add_trace(go.Scatter(
                x=sub["slot"], y=sub["kwh_plot"],
                mode="lines",
                name=f"{flow_name} ({name_suffix})",
                line=dict(dash=dash, width=width)
            ))

    add_lines(both[both["scenario"] == "before"], "before", dash="dot", width=2)
    add_lines(both[both["scenario"] == "after"],  "after",  dash=None,   width=3)

    fig.update_layout(
        title=title,
        xaxis_title="Time of Day",
        yaxis_title="kWh per interval",
        hovermode="x unified",
        legend_title_text="Series",
    )
    fig.update_xaxes(type="category", tickangle=-45)
    fig.add_hline(y=0, line_color="black", line_width=0.6)

    return fig

# --- Usage example ---
# Assuming you already have `result = run(df, ev=..., pv=..., battery=..., plan=...)`:
fig = plot_avg_profile_before_after(result.df_before, result.df_after, show_net=True)
fig.show()


In [31]:

# # 1) PV at night should be (very close to) zero
# pv = apply_pv(idx, pv_cfg, interval_h=... )
# night = (idx.tz_convert(idx.tz).hour >= 22) | (idx.tz_convert(idx.tz).hour < 5)
# float(pv[night].sum())
# # Expect ~0.0; if not, the PV shape is misaligned.

# # 2) Hour-of-day profile to spot the bump
# pv_by_h = pv.groupby(idx.tz_convert(idx.tz).hour).mean()
# print(pv_by_h)  # If hour 0 (midnight) is nonzero, that’s the culprit.


In [32]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

try:
    import meterlogic as ml
except ImportError:
    import meterdatalogic as ml

def plot_profile_overlay_with_delta(df_before: pd.DataFrame, df_after: pd.DataFrame,
                                    title="Average 24-Hour Profile — Baseline vs Scenario"):
    # Average profiles by 30-min slot (or whatever cadence)
    prof_b = ml.transform.profile24(df_before)  # slot + flows
    prof_a = ml.transform.profile24(df_after)

    # Keep consistent slot ordering
    slots = prof_b["slot"]
    prof_a = prof_a.set_index("slot").reindex(slots).reset_index()

    # Get import/export (missing -> zeros)
    def col(df, name):
        return df[name] if name in df.columns else pd.Series(0.0, index=df.index)

    imp_b = col(prof_b, "grid_import")
    imp_a = col(prof_a, "grid_import")
    exp_b = -col(prof_b, "grid_export_solar")  # negative for plotting
    exp_a = -col(prof_a, "grid_export_solar")

    # Build figure
    fig = go.Figure()

    # Baseline import (thin grey)
    fig.add_trace(go.Scatter(x=slots, y=imp_b, name="Import (before)",
                             mode="lines", line=dict(color="gray", width=2, dash="dot")))

    # Scenario import (bold)
    fig.add_trace(go.Scatter(x=slots, y=imp_a, name="Import (after)",
                             mode="lines", line=dict(width=3)))

    # Shade delta between before and after (import)
    fig.add_trace(go.Scatter(
        x=pd.concat([slots, slots[::-1]]),
        y=pd.concat([imp_a, imp_b[::-1]]),
        fill="toself",
        fillcolor="rgba(0, 150, 255, 0.15)",
        line=dict(color="rgba(0,0,0,0)"),
        name="Δ Import (after − before)",
        hoverinfo="skip",
        showlegend=True
    ))

    # Exports below zero (optional but nice)
    fig.add_trace(go.Scatter(x=slots, y=exp_b, name="Export (before)",
                             mode="lines", line=dict(color="orange", width=2, dash="dot")))
    fig.add_trace(go.Scatter(x=slots, y=exp_a, name="Export (after)",
                             mode="lines", line=dict(color="orange", width=3)))

    fig.update_layout(
        title=title,
        xaxis_title="Time of day",
        yaxis_title="kWh per interval",
        hovermode="x unified",
        legend_title_text="Series",
    )
    fig.update_xaxes(type="category", tickangle=-45)
    fig.add_hline(y=0, line_color="black", line_width=0.6)
    return fig

# Usage:
fig = plot_profile_overlay_with_delta(result.df_before, result.df_after)
fig.show()


In [33]:
import numpy as np
import pandas as pd
import plotly.express as px

def plot_profile_bars_before_after(df_before: pd.DataFrame, df_after: pd.DataFrame,
                                   title="Average 24-Hour Profile — Bars Before vs After"):
    prof_b = ml.transform.profile24(df_before)
    prof_a = ml.transform.profile24(df_after)

    # Tidy and mark scenario
    def tidy(prof, label):
        # import
        imp = prof[["slot"]].copy()
        imp["flow"] = "grid_import"
        imp["kwh"] = prof.get("grid_import", pd.Series(0.0, index=prof.index))
        imp["scenario"] = label
        # export (neg)
        exp = prof[["slot"]].copy()
        exp["flow"] = "grid_export_solar"
        exp["kwh"] = -prof.get("grid_export_solar", pd.Series(0.0, index=prof.index))
        exp["scenario"] = label
        return pd.concat([imp, exp], ignore_index=True)

    tb = tidy(prof_b, "before")
    ta = tidy(prof_a, "after")
    both = pd.concat([tb, ta], ignore_index=True)

    fig = px.bar(
        both, x="slot", y="kwh", color="scenario", facet_row="flow",
        barmode="group", title=title,
        labels={"slot": "Time of day", "kwh": "kWh per interval"}
    )
    fig.update_xaxes(type="category", tickangle=-45)
    fig.add_hline(y=0, line_color="black", line_width=0.6, row=1, col=1)
    fig.add_hline(y=0, line_color="black", line_width=0.6, row=2, col=1)
    return fig

# Usage:
fig = plot_profile_bars_before_after(result.df_before, result.df_after)
fig.show()


In [34]:
import plotly.graph_objects as go
import pandas as pd

# Convert profile24 lists to DataFrames if they come as lists of dicts
df_before = pd.DataFrame(result.summary_before["profile24"])
df_after = pd.DataFrame(result.summary_after["profile24"])

# Ensure slot (HH:MM) column is sorted in time order
df_before = df_before.sort_values("slot")
df_after = df_after.sort_values("slot")

# Create Plotly figure
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_before["slot"],
    y=df_before["grid_import"],
    mode="lines+markers",
    name="Before Scenario",
    line=dict(color="royalblue"),
    marker=dict(size=4)
))

fig.add_trace(go.Scatter(
    x=df_after["slot"],
    y=df_after["grid_import"],
    mode="lines+markers",
    name="After Scenario",
    line=dict(color="orange"),
    marker=dict(size=4)
))

fig.update_layout(
    title="Average Daily Profile – Before vs After Scenario",
    xaxis_title="Time of Day",
    yaxis_title="Grid Import (kWh per 30-min interval)",
    template="plotly_white",
    hovermode="x unified",
    legend=dict(x=0.02, y=0.98)
)

fig.show()
